<a href="https://colab.research.google.com/github/leonardobora/pratica-aprendizado-de-maquina/blob/main/Task02_Heart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Model, utils
from sklearn.model_selection import train_test_split
import SimpleITK as sitk
from skimage.transform import resize

# 1. Configuração do Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Caminhos do dataset (ajuste conforme sua estrutura)
dataset_path = '/content/drive/MyDrive/Medical_Decathlon/Task02_Heart'
images_dir = os.path.join(dataset_path, 'imagesTr')
masks_dir = os.path.join(dataset_path, 'labelsTr')

# 3. Funções de carregamento melhoradas
def load_volume(filepath):
    """Carrega volume 3D e retorna array numpy com orientação correta"""
    img = nib.load(filepath)
    data = img.get_fdata()
    data = np.transpose(data, (2, 1, 0))  # Corrige orientação
    return data

def preprocess_volume(volume, target_size=(128, 128)):
    """Pré-processa volume 3D com preservação de proporções"""
    processed = []
    for slice_idx in range(volume.shape[0]):
        # Redimensionamento inteligente mantendo aspect ratio
        slice_img = volume[slice_idx, :, :]

        # Normalização baseada em percentis (melhor para imagens médicas)
        p2 = np.percentile(slice_img, 2)
        p98 = np.percentile(slice_img, 98)
        slice_norm = np.clip((slice_img - p2) / (p98 - p2 + 1e-8), 0, 1)

        # Redimensionamento com preservação de proporções
        slice_resized = resize(slice_norm, target_size,
                              order=1, mode='constant',
                              preserve_range=True, anti_aliasing=True)
        processed.append(slice_resized)
    return np.array(processed)

# 4. Carregamento correto do dataset multiclasse
X, y = [], []

# Listar apenas arquivos de imagens válidos
image_files = [f for f in sorted(os.listdir(images_dir))
              if f.endswith('.nii.gz') and '_0000' in f]

for file in image_files:
    img_path = os.path.join(images_dir, file)
    mask_path = os.path.join(masks_dir, file.replace('_0000', ''))

    # Verificar se máscara correspondente existe
    if not os.path.exists(mask_path):
        print(f"Aviso: Máscara não encontrada para {file}")
        continue

    # Carregar volumes
    img_volume = load_volume(img_path)
    mask_volume = load_volume(mask_path)

    # Pré-processar
    img_processed = preprocess_volume(img_volume)
    mask_processed = preprocess_volume(mask_volume, target_size=(128, 128))

    # Preservar informações multiclasse
    X.append(img_processed)
    y.append(mask_processed)

# 5. Preparação dos dados
X = np.concatenate(X, axis=0)[..., np.newaxis]  # Add channel dimension
y = np.concatenate(y, axis=0)

# Converter máscaras para one-hot encoding (3 classes: fundo + 2 estruturas cardíacas)
y = utils.to_categorical(y, num_classes=3)

# 6. Divisão dos dados mantendo volumes intactos
# Estratégia: dividir por volume em vez de por fatias
vol_sizes = [vol.shape[0] for vol in X]
train_idx, val_idx = train_test_split(
    range(len(vol_sizes)),
    test_size=0.2,
    random_state=42
)

# Criar máscaras de seleção
train_mask = np.zeros(len(X), dtype=bool)
val_mask = np.zeros(len(X), dtype=bool)

start = 0
for i, size in enumerate(vol_sizes):
    end = start + size
    if i in train_idx:
        train_mask[start:end] = True
    else:
        val_mask[start:end] = True
    start = end

X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val = X[val_mask], y[val_mask]

print(f"Dados de treino: {X_train.shape[0]} fatias")
print(f"Dados de validação: {X_val.shape[0]} fatias")
print(f"Distribuição de classes: {np.unique(np.argmax(y_train, axis=-1), return_counts=True)}")

# 7. Visualização de amostra
def plot_sample(idx):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))

    ax[0].imshow(X_train[idx].squeeze(), cmap='gray')
    ax[0].set_title('Imagem de entrada')

    mask = np.argmax(y_train[idx], axis=-1)
    ax[1].imshow(mask, cmap='jet')
    ax[1].set_title('Máscara verdadeira')

    # Mostrar estruturas separadas
    structures = []
    for i in range(1, 3):  # Classes 1 e 2
        structures.append(mask == i)

    ax[2].imshow(structures[0], cmap='Reds', alpha=0.5)
    ax[2].imshow(structures[1], cmap='Blues', alpha=0.5)
    ax[2].set_title('Estruturas cardíacas')

    plt.tight_layout()
    plt.show()

plot_sample(np.random.randint(0, len(X_train)))

ModuleNotFoundError: No module named 'SimpleITK'